In [1]:
import time
import random
import os
import re
import pandas as pd
import numpy as np
from datetime import datetime
from elasticsearch import Elasticsearch 

In [2]:
import mysql.connector

# 日期处理

In [3]:
def date_convert(dot_date_str):
    '''xxxx.xx.xx --> xxxx-xx-xx
    then pack into json for es
    '''
    y,m,d = dot_date_str.split('.')
    new_date = '{year}-{month}-{date}'.format(year=y,month=m,date=d)
    return new_date
#     return {new_date:{'type':'date','formate':'yyyy-mm-dd'}}

In [4]:
def date_autofill(dot_date_str):
    '''xxxx --> xxxx-01-01
    then pack into json for es
    '''
    new_date = '{year}-{month}-{date}'.format(year=dot_date_str,month='01',date='01')
    return new_date

# 读取库中paper数据

In [5]:
head = 'P_Title, P_Author, P_Publication, P_Organ, P_year, P_Keyword, \
    P_Summary, P_Keyword_seg, P_Title_seg,\
    P_Summary_seg, P_URL, P_Fields, P_Fields_two,P_References, P_Pagecount, P_Page, P_Language,\
    P_Download_num, P_Citation_num,P_Vector,P_Volume, P_Issue,P_Issn,P_Isbn, P_Doi,\
    P_Red1, P_Red2, P_Red3, P_Red4, P_Red5'

In [6]:
head_columns = [i.strip() for i in head.split(',')]

In [8]:
cnx = mysql.connector.connect(user='root', password='ir@DUT#2018',
                              host='10.8.128.205',port='23306')

cursor = cnx.cursor()

# 获取paper表头

In [9]:
cursor.execute("select * from information_schema.columns where table_name='paper_table'")

In [10]:
sql_table_columns = cursor.fetchall()

In [11]:
sql_table_columns_name = [i[3] for i in sql_table_columns]

In [12]:
sql_table_columns_name

['P_ID',
 'P_Title',
 'P_Author',
 'P_Publication',
 'P_Organ',
 'P_year',
 'P_Keyword',
 'P_Summary',
 'P_Keyword_seg',
 'P_Title_seg',
 'P_Summary_seg',
 'P_URL',
 'P_Fields',
 'P_Fields_two',
 'P_References',
 'P_Pagecount',
 'P_Page',
 'P_Language',
 'P_Download_num',
 'P_Citation_num',
 'P_Vector',
 'P_Volume',
 'P_Issue',
 'P_Issn',
 'P_Isbn',
 'P_Doi',
 'P_Red1',
 'P_Red2',
 'P_Red3',
 'P_Red4',
 'P_Red5']

# 获取paper表总容量

In [13]:
cursor.execute("select table_rows from information_schema.tables where table_name='paper_table'")

In [14]:
sql_table_rows = cursor.fetchall()

In [15]:
sql_table_rows = sql_table_rows[0][0]

In [16]:
def data_to_json(bio_data_atom):
    data = bio_data_atom[1:]
    data_dict = {k:data[num] for num,k in enumerate(head_columns)}
    data_dict['P_year'] = date_autofill(data_dict['P_year'])
    return data_dict

# 构建索引

In [18]:
from elasticsearch.helpers import bulk

In [19]:
es = Elasticsearch(hosts='10.8.128.205',port=49200,timeout=30, max_retries=10, retry_on_timeout=True)

# 每次查询limit数目的数值 同时批量插入paper

In [ ]:
limit = 5000
offset = 0
for epoch in range(int(sql_table_rows/limit)):
    print('[{0}] Epoch:{1}'.format(time.ctime(),epoch))
    base_fetch_sql = 'select * from {dbName}.{tableName} limit {limitVolume} offset {offsetVolume}'
    base_fetch_sql = base_fetch_sql.format(dbName='bioDB',tableName='paper_table',limitVolume=limit,offsetVolume=offset)
    cursor.execute(base_fetch_sql)
    sql_data = cursor.fetchall()
    sql_dict = [data_to_json(i) for i in sql_data]
    bulk(es,sql_dict,index='paper-year-index',raise_on_error=True)
    offset += limit - 1
# 最后一桶
base_fetch_sql = 'select * from {dbName}.{tableName} limit {limitVolume} offset {offsetVolume}'
base_fetch_sql = base_fetch_sql.format(dbName='bioDB',tableName='paper_table',limitVolume=limit,offsetVolume=offset)
cursor.execute(base_fetch_sql)
sql_data = cursor.fetchall()
sql_dict = [data_to_json(i) for i in sql_data]
bulk(es,sql_dict,index='paper-year-index',raise_on_error=True)

In [50]:
cursor.close()

True

In [45]:
offset

1274745

In [46]:
sql_table_rows

1278737

# 读取库中author数据

In [51]:
head = 'A_Cname, A_Ename, A_Normalized_name, A_Organ, A_Position, A_Fields , A_Pubs_num,\
    A_Citation_num, A_Index, A_Research_interest, A_Focus_Fields, A_Paper_ID, A_Co_ID, A_Co_name, A_Vector,\
    A_Red1, A_Red2, A_Red3, A_Red4, A_Red5'

In [52]:
head_columns = [i.strip() for i in head.split(',')]

In [56]:
cnx = mysql.connector.connect(user='root', password='ir@DUT#2018',
                              host='10.8.128.205',port='23306')

cursor = cnx.cursor()

# 获取paper表头

In [57]:
cursor.execute("select * from information_schema.columns where table_name='author_table'")

In [58]:
sql_table_columns = cursor.fetchall()

In [59]:
sql_table_columns_name = [i[3] for i in sql_table_columns]

In [60]:
sql_table_columns_name

['A_ID',
 'A_Cname',
 'A_Ename',
 'A_Normalized_name',
 'A_Organ',
 'A_Position',
 'A_Fields',
 'A_Pubs_num',
 'A_Citation_num',
 'A_Index',
 'A_Research_interest',
 'A_Focus_Fields',
 'A_Paper_ID',
 'A_Co_ID',
 'A_Co_name',
 'A_Vector',
 'A_Red1',
 'A_Red2',
 'A_Red3',
 'A_Red4',
 'A_Red5']

# 获取paper表总容量

In [61]:
cursor.execute("select table_rows from information_schema.tables where table_name='author_table'")

In [62]:
sql_table_rows = cursor.fetchall()

In [63]:
sql_table_rows = sql_table_rows[0][0]

In [70]:
def data_to_json_author(data_atom):
    data = data_atom[1:]
    data_dict = {k:data[num] for num,k in enumerate(head_columns)}
    return data_dict

# 每次查询limit数目的数值 同时批量插入author

In [73]:
from elasticsearch.helpers import bulk

In [74]:
es = Elasticsearch(hosts='10.8.128.205',port=49200,timeout=30, max_retries=10, retry_on_timeout=True)

In [75]:
limit = 10000
offset = 0
for epoch in range(int(sql_table_rows/limit)):
    print('[{0}] Epoch:{1}'.format(time.ctime(),epoch))
    base_fetch_sql = 'select * from {dbName}.{tableName} limit {limitVolume} offset {offsetVolume}'
    base_fetch_sql = base_fetch_sql.format(dbName='bioDB',tableName='author_table',limitVolume=limit,offsetVolume=offset)
    cursor.execute(base_fetch_sql)
    sql_data = cursor.fetchall()
    sql_dict = [data_to_json_author(i) for i in sql_data]
    bulk(es,sql_dict,index='paper-author-index',raise_on_error=True)
    offset += limit - 1
# 最后一桶
base_fetch_sql = 'select * from {dbName}.{tableName} limit {limitVolume} offset {offsetVolume}'
base_fetch_sql = base_fetch_sql.format(dbName='bioDB',tableName='author_table',limitVolume=limit,offsetVolume=offset)
cursor.execute(base_fetch_sql)
sql_data = cursor.fetchall()
sql_dict = [data_to_json_author(i) for i in sql_data]
bulk(es,sql_dict,index='paper-author-index',raise_on_error=True)

[Wed Jul 17 01:13:56 2019] Epoch:0
[Wed Jul 17 01:14:02 2019] Epoch:1
[Wed Jul 17 01:14:07 2019] Epoch:2
[Wed Jul 17 01:14:12 2019] Epoch:3
[Wed Jul 17 01:14:17 2019] Epoch:4
[Wed Jul 17 01:14:22 2019] Epoch:5
[Wed Jul 17 01:14:27 2019] Epoch:6
[Wed Jul 17 01:14:31 2019] Epoch:7
[Wed Jul 17 01:14:36 2019] Epoch:8
[Wed Jul 17 01:14:41 2019] Epoch:9
[Wed Jul 17 01:14:46 2019] Epoch:10
[Wed Jul 17 01:14:51 2019] Epoch:11
[Wed Jul 17 01:14:56 2019] Epoch:12
[Wed Jul 17 01:15:01 2019] Epoch:13
[Wed Jul 17 01:15:06 2019] Epoch:14
[Wed Jul 17 01:15:11 2019] Epoch:15
[Wed Jul 17 01:15:16 2019] Epoch:16
[Wed Jul 17 01:15:21 2019] Epoch:17
[Wed Jul 17 01:15:26 2019] Epoch:18
[Wed Jul 17 01:15:32 2019] Epoch:19
[Wed Jul 17 01:15:37 2019] Epoch:20
[Wed Jul 17 01:15:42 2019] Epoch:21
[Wed Jul 17 01:15:47 2019] Epoch:22
[Wed Jul 17 01:15:52 2019] Epoch:23
[Wed Jul 17 01:15:57 2019] Epoch:24
[Wed Jul 17 01:16:02 2019] Epoch:25
[Wed Jul 17 01:16:08 2019] Epoch:26
[Wed Jul 17 01:16:13 2019] Epoch:27
[W

(10000, [])

# 设置ES状态为可写（触发硬盘存储超限后集群会自动配置所有索引为只读）

In [42]:
import json

import requests

headers = {
    'Accept': 'application/json, text/javascript, */*; q=0.01',
    'Content-Type': 'application/json',
    'Origin': 'chrome-extension://ffmkiejjmecolpfloofpjologoblkegm',
    'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/75.0.3770.100 Safari/537.36'
}

unlock_dict = {
"index.blocks.read_only_allow_delete": "false"
}

ppp = requests.put('http://10.8.128.205:49200/paper-year-index/_settings',data=json.dumps(unlock_dict),headers=headers)

In [43]:
ppp.text

'{"acknowledged":true}'

# 论文检索对象

In [81]:
ess = esPatentRetrieval(host='10.8.128.205',port=29200)

In [ ]:
ess.Retrieval('机器人','2000-01-22','2010-01-22',volume=100)